# Libraries

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr

from dhydro_utils import (
    run_simulations_mesh,
    create_raw_dataset_folder,
    create_mesh_dhydro,
)
from graph_creation import Mesh, plot_faces, save_polygon_to_file, find_closest_nodes

# Run simulations

## Dike ring

In [ ]:
model_folder = "C:\\Users\\rbentivoglio\\Documents\\PhD\\My PhD\\D-HYDRO\\dijkring_15.dsproj_data\\FlowFM\\input\\dflowfm"
save_folder = "raw_datasets_dk15"
polygon_file = "dijkring_15.pol"
DEM_file = model_folder + "/DEM_25m_15.xyz"

In [ ]:
from shapely.geometry import Polygon
from shapely import geometry
import fiona
import numpy as np

shapefile_path = "C:/Users/rbentivoglio/Documents/PhD/GeoData/Nederland/dijkring_15.shp"

with fiona.open(shapefile_path, "r") as shapefile:
    shape = shapefile[0]

    line = geometry.LineString(shape["geometry"].coordinates[0])
    polygon = geometry.Polygon(line)

polygon = Polygon(np.array(shape.geometry.coordinates)[0])
polygon = polygon.simplify(tolerance=300, preserve_topology=True)

save_polygon_to_file(polygon, polygon_file)

In [ ]:
plt.plot(np.array(polygon.boundary.xy)[0], np.array(polygon.boundary.xy)[1])

# project polygon boundary to a line
line = geometry.LineString(polygon.boundary.coords)

# uniform points along the line
distance = 8700  # 6500
points = [
    line.interpolate(i * distance + 2000).xy for i in range(int(line.length / distance))
]
points = np.array(points).squeeze()

# plot points
print("Number of points: ", len(points))
plt.plot(points[:, 0], points[:, 1], "x");

In [ ]:
mesh = create_mesh_dhydro(polygon_file, 4)

boundary_edges = np.where((mesh.edge_faces.reshape(-1, 2) == -1).sum(1) == 1)[0]
boundary_nodes = mesh.mesh_nodes[
    mesh.edge_nodes.reshape(-1, 2)[boundary_edges]
].reshape(-1, 2)

In [ ]:
np.random.seed(42)

total_time = 3600 * 24 * 4
time_resolution = 3600
min_discharge = 0
peak_value = (np.random.rand(len(points))) * 250 + 750
Ln = 10
Q = 0.3

time_steps = int(total_time / time_resolution) + 1
time_x = np.linspace(0.25, 0.8, time_steps)
time_hydrograph = time_x - time_x.min()
time_hydrograph = time_hydrograph / time_hydrograph.max() * total_time

for i, point in enumerate(points):
    boundary_edge = find_closest_nodes(boundary_nodes, point, top_n=3)
    coords = boundary_nodes[boundary_edge]

    F = time_x**2
    y = (
        F
        * (Ln - 1)
        / np.sqrt((Ln * F - 1) ** 2 + F * (F - 1) ** 2 * (Ln - 1) ** 2 * Q**2)
    )
    y = y / y.max() * (peak_value[i] - min_discharge) + min_discharge

    hydrograph = (time_hydrograph, y)
    df = run_simulations_mesh(
        1,
        model_folder,
        save_folder,
        start_sim=101 + i,
        DEM_file=DEM_file,
        polygon_file=polygon_file,
        breach_coords=coords[1:],
        number_of_multiscales=4,
        random_hydrograph=False,
        hydrograph=hydrograph,
    )

df

## Synthetic dataset

In [ ]:
model_folder = "C:\\Users\\rbentivoglio\\Documents\\PhD\\My PhD\\D-HYDRO\\SWEGNN_mesh.dsproj_data\\FlowFM\\input\\dflowfm"
save_folder = "raw_datasets_mesh"

create_raw_dataset_folder(save_folder)

In [ ]:
df = run_simulations_mesh(
    1,
    model_folder,
    save_folder,
    start_sim=91,
    # DEM_file=DEM_file, polygon_file=polygon_file, breach_coords=coords[1:],
    noise_octave=(1, 5),
    DEM_multiplier=(0.5, 5),
    slope_multiplier=(0, 0.005),
    num_vertices_polygon=(24, 28),
    number_of_multiscales=4,
    ellipticality=(1, 2),
    grid_size=83,
    random_hydrograph=True,
    min_discharge=0,
    peak=(150, 300),
    shape=(0, 2),
)

# Visualize results

In [ ]:
mesh = Mesh()
_id = 106
nc_file = f"{save_folder}/Simulations/output_{_id}_map.nc"
mesh._import_from_map_netcdf(nc_file)
mesh._get_derived_attributes()
mesh._import_DEM(f"{save_folder}\\DEM\\DEM_{_id}.xyz")
mesh

In [ ]:
nc_dataset = xr.open_dataset(nc_file)

waterdepth = nc_dataset["mesh2d_waterdepth"].data

fig, axs = plt.subplots(1, 2, figsize=(12, 5))
plot_faces(
    mesh,
    ax=axs[1],
    face_value=waterdepth[:, -1],
    cmap="Blues",
    edgecolor="k",
    linewidths=0.1,
)

cbar = plt.colorbar(axs[1].collections[0], ax=axs[1], orientation="vertical")
# cbar.set_label('Color')

plot_faces(
    mesh,
    ax=axs[0],
    face_value=mesh.DEM,
    cmap="terrain",
    edgecolor="k",
    linewidths=0.1,
)
cbar = plt.colorbar(axs[0].collections[0], ax=axs[0], orientation="vertical")